# CIFO - Pipeline Completo: Execução e Análise de Algoritmos de Otimização

Este notebook integra todo o processo desde a execução dos algoritmos até à visualização e análise dos resultados.

## 1. Configuração do Ambiente e Importações

In [1]:
# Importações necessárias
import random
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
import os
from datetime import datetime
import warnings
from collections import defaultdict

# Configurar matplotlib para notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 10)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

# Suprimir avisos específicos
warnings.filterwarnings("ignore", message="scipy.stats.shapiro: Input data has range zero.*")
warnings.filterwarnings("ignore", message="No artists with labels found to put in legend.*")

# Importar módulos do projeto
from solution import LeagueSolution, LeagueHillClimbingSolution
from evolution import hill_climbing, simulated_annealing
from operators import (
    selection_tournament,
    selection_ranking,
    selection_boltzmann,
    crossover_one_point,
    crossover_uniform,
    mutate_swap, 
    mutate_swap_constrained,
    genetic_algorithm
)
from fitness_counter import FitnessCounter

## 2. Implementação de Funções Auxiliares

In [2]:
# Implementar crossover de dois pontos (não existe em operators.py)
def two_point_crossover(parent1, parent2):
    """
    Two-point crossover: creates a child by taking portions from each parent.
    
    Args:
        parent1 (LeagueSolution): First parent solution
        parent2 (LeagueSolution): Second parent solution
        
    Returns:
        LeagueSolution: A new solution created by crossover
    """
    cut1 = random.randint(1, len(parent1.repr) - 3)
    cut2 = random.randint(cut1 + 1, len(parent1.repr) - 2)
    child_repr = parent1.repr[:cut1] + parent2.repr[cut1:cut2] + parent1.repr[cut2:]
    
    return LeagueSolution(
        repr=child_repr,
        num_teams=parent1.num_teams,
        team_size=parent1.team_size,
        max_budget=parent1.max_budget,
        players=parent1.players
    )

## 3. Configuração dos Parâmetros de Execução

In [16]:
# Configurar seed para reprodutibilidade
random.seed(42)
np.random.seed(42)

# Configurar parâmetros gerais
NUM_RUNS = 30  # Número de execuções para cada algoritmo
MAX_EVALUATIONS = 10000  # Número máximo de avaliações de função
POPULATION_SIZE = 100  # Tamanho da população para algoritmos genéticos
MAX_GENERATIONS = 100  # Número máximo de gerações para algoritmos genéticos

## 4. Carregamento dos Dados dos Jogadores

In [17]:
# Carregar dados dos jogadores
players_df = pd.read_csv('players.csv', encoding='utf-8',sep=';', index_col=0)

# Mostrar primeiras linhas
print("Dados dos jogadores:")
display(players_df.head())

# Renomear colunas para compatibilidade com o código
column_mapping = {
    'Salary (€M)': 'Salary'
}
players_df = players_df.rename(columns=column_mapping)

# Converter DataFrame para lista de dicionários
players_list = players_df.to_dict('records')

# Configurar contador de fitness
fitness_counter = FitnessCounter()

Dados dos jogadores:


,Name,Position,Skill,Salary (€M)
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95


## 5. Configuração dos Algoritmos

In [18]:
# Configurar algoritmos
configs = {
    # Algoritmos base
    'HC_Standard': {
        'algorithm': 'Hill Climbing',
    },
    'SA_Standard': {
        'algorithm': 'Simulated Annealing',
    },
    'GA_Tournament_OnePoint': {
        'algorithm': 'Genetic Algorithm',
        'selection': 'Tournament',
        'crossover': 'One Point',
        'mutation_rate': 1.0/35,  # 1.0/len(players)
        'elitism_percent': 0.1,   # 10%
        'population_size': 100,
        'use_valid_initial': False,
        'use_repair': False,
    },
    'GA_Tournament_TwoPoint': {
        'algorithm': 'Genetic Algorithm',
        'selection': 'Tournament',
        'crossover': 'Two Point',
        'mutation_rate': 1.0/35,
        'elitism_percent': 0.1,
        'population_size': 100,
        'use_valid_initial': False,
        'use_repair': False,
    },
    'GA_Rank_Uniform': {
        'algorithm': 'Genetic Algorithm',
        'selection': 'Rank',
        'crossover': 'Uniform',
        'mutation_rate': 1.0/35,
        'elitism_percent': 0.1,
        'population_size': 100,
        'use_valid_initial': False,
        'use_repair': False,
    },
    'GA_Boltzmann_TwoPoint': {
        'algorithm': 'Genetic Algorithm',
        'selection': 'Boltzmann',
        'crossover': 'Two Point',
        'mutation_rate': 1.0/35,
        'elitism_percent': 0.1,
        'population_size': 100,
        'use_valid_initial': False,
        'use_repair': False,
    },
    'GA_Hybrid': {
        'algorithm': 'Genetic Algorithm Hybrid',
        'selection': 'Tournament',
        'crossover': 'Two Point',
        'mutation_rate': 1.0/35,
        'elitism_percent': 0.1,
        'population_size': 100,
        'use_valid_initial': False,
        'use_repair': False,
    }
}

# Mostrar configurações
print("Configurações dos algoritmos:")
for config_name, config in configs.items():
    print(f"\n{config_name}:")
    for key, value in config.items():
        print(f"  {key}: {value}")

Configurações dos algoritmos:

HC_Standard:
  algorithm: Hill Climbing

SA_Standard:
  algorithm: Simulated Annealing

GA_Tournament_OnePoint:
  algorithm: Genetic Algorithm
  selection: Tournament
  crossover: One Point
  mutation_rate: 0.02857142857142857
  elitism_percent: 0.1
  population_size: 100
  use_valid_initial: False
  use_repair: False

GA_Tournament_TwoPoint:
  algorithm: Genetic Algorithm
  selection: Tournament
  crossover: Two Point
  mutation_rate: 0.02857142857142857
  elitism_percent: 0.1
  population_size: 100
  use_valid_initial: False
  use_repair: False

GA_Rank_Uniform:
  algorithm: Genetic Algorithm
  selection: Rank
  crossover: Uniform
  mutation_rate: 0.02857142857142857
  elitism_percent: 0.1
  population_size: 100
  use_valid_initial: False
  use_repair: False

GA_Boltzmann_TwoPoint:
  algorithm: Genetic Algorithm
  selection: Boltzmann
  crossover: Two Point
  mutation_rate: 0.02857142857142857
  elitism_percent: 0.1
  population_size: 100
  use_valid_in

## 6. Implementação dos Algoritmos

In [23]:
# Função para executar Hill Climbing
def run_hill_climbing(players, max_evaluations):
    solution = LeagueSolution(players)
    
    # Iniciar contagem de fitness
    fitness_counter.reset()
    solution.set_fitness_counter(fitness_counter)
    
    best_fitness = solution.fitness()
    history = [best_fitness]
    
    while fitness_counter.get_count() < max_evaluations:
        # Gerar vizinho
        neighbor = deepcopy(solution)
        idx = random.randint(0, len(neighbor.repr) - 1)
        neighbor.repr[idx] = random.randint(0, neighbor.num_teams - 1)
        
        neighbor_fitness = neighbor.fitness()
        
        # Aceitar se melhor
        if neighbor_fitness < best_fitness:  # Menor é melhor
            solution = neighbor
            best_fitness = neighbor_fitness
        
        history.append(best_fitness)
    
    return solution, history, fitness_counter.get_count()

# Função para executar Simulated Annealing
def run_simulated_annealing(players, max_evaluations):
    solution = LeagueSolution(players)
    
    # Iniciar contagem de fitness
    fitness_counter.reset()
    solution.set_fitness_counter(fitness_counter)
    
    best_solution = deepcopy(solution)
    current_fitness = solution.fitness()
    best_fitness = current_fitness
    
    history = [best_fitness]
    
    # Parâmetros do SA
    initial_temp = 100.0
    final_temp = 0.1
    alpha = 0.95
    
    current_temp = initial_temp
    
    while fitness_counter.get_count() < max_evaluations and current_temp > final_temp:
        # Gerar vizinho
        neighbor = deepcopy(solution)
        idx = random.randint(0, len(neighbor.repr) - 1)
        neighbor.repr[idx] = random.randint(0, neighbor.num_teams - 1)
        
        neighbor_fitness = neighbor.fitness()
        
        # Calcular delta
        delta = neighbor_fitness - current_fitness
        
        # Aceitar se melhor ou com probabilidade baseada na temperatura
        if delta < 0 or random.random() < np.exp(-delta / current_temp):
            solution = neighbor
            current_fitness = neighbor_fitness
            
            # Atualizar melhor solução se necessário
            if current_fitness < best_fitness:
                best_solution = deepcopy(solution)
                best_fitness = current_fitness
        
        history.append(best_fitness)
        
        # Resfriar
        current_temp *= alpha
    
    return best_solution, history, fitness_counter.get_count()

# Função para executar Genetic Algorithm
def run_genetic_algorithm(players, config, max_evaluations):
    # Iniciar contagem de fitness
    fitness_counter.reset()
    
    # Configurar seleção
    if config['selection'] == 'Tournament':
        selection_op = selection_tournament
    elif config['selection'] == 'Rank':
        selection_op = selection_ranking
    elif config['selection'] == 'Boltzmann':
        selection_op = selection_boltzmann
    else:
        raise ValueError(f"Seleção não suportada: {config['selection']}")
    
    # Configurar crossover
    if config['crossover'] == 'One Point':
        crossover_op = crossover_one_point
    elif config['crossover'] == 'Two Point':
        crossover_op = two_point_crossover
    elif config['crossover'] == 'Uniform':
        crossover_op = crossover_uniform
    else:
        raise ValueError(f"Crossover não suportado: {config['crossover']}")
    
    # Configurar mutação
    mutation_op = mutate_swap
    
    # Configurar operador de reparo (se necessário)
    repair_op = None
    if config.get('use_repair', False):
        def repair_operator(solution):
            # Implementação simples de reparo: tenta corrigir soluções inválidas
            # ajustando a distribuição de jogadores por posição e orçamento
            if solution.is_valid():
                return solution
            
            # Obter estatísticas das equipes
            teams = solution.get_teams()
            
            # Verificar e corrigir distribuição de posições
            for team_idx, team in enumerate(teams):
                positions = {"GK": 0, "DEF": 0, "MID": 0, "FWD": 0}
                for player in team:
                    positions[player["Position"]] += 1
                
                # Se a distribuição estiver incorreta, tentar corrigir
                if positions != {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}:
                    # Implementação simplificada: apenas retorna a solução original
                    # Uma implementação real seria mais complexa
                    pass
            
            return solution
        
        repair_op = repair_operator
    
    # Configurar local search para GA híbrido
    local_search = None
    if config['algorithm'] == 'Genetic Algorithm Hybrid':
        local_search = {
            'operator': 'hill_climbing',
            'probability': 0.1,
            'iterations': 10
        }
    
    # Executar GA
    best_solution, best_fitness, history = genetic_algorithm(
        players=players,
        population_size=config['population_size'],
        max_generations=MAX_GENERATIONS,
        selection_operator=selection_op,
        crossover_operator=crossover_op,
        crossover_rate=0.8,
        mutation_operator=mutation_op,
        mutation_rate=config['mutation_rate'],
        elitism=config['elitism_percent'] > 0,
        elitism_size=int(config['population_size'] * config['elitism_percent']),
        local_search=local_search,
        fitness_counter=fitness_counter,
        max_evaluations=max_evaluations,
        verbose=False
    )
    
    return best_solution, history, fitness_counter.get_count()

## 7. Execução dos Experimentos

In [24]:
# Função para executar um experimento completo
def run_experiment(config_name, config, players, num_runs, max_evaluations):
    results = []
    all_history = []
    
    for run in range(num_runs):
        print(f"Executando {config_name}, run {run+1}/{num_runs}...")
        
        start_time = time.time()
        
        try:
            if config['algorithm'] == 'Hill Climbing':
                best_solution, history, evaluations = run_hill_climbing(players, max_evaluations)
            elif config['algorithm'] == 'Simulated Annealing':
                best_solution, history, evaluations = run_simulated_annealing(players, max_evaluations)
            elif 'Genetic Algorithm' in config['algorithm']:
                best_solution, history, evaluations = run_genetic_algorithm(players, config, max_evaluations)
            else:
                raise ValueError(f"Algoritmo não suportado: {config['algorithm']}")
            
            end_time = time.time()
            execution_time = end_time - start_time
            
            # Registrar resultados
            results.append({
                'Configuration': config_name,
                'Run': run + 1,
                'Best Fitness': best_solution.fitness(),
                'Function Evaluations': evaluations,
                'Runtime (s)': execution_time,
                'Valid': best_solution.is_valid()
            })
            
            all_history.append(history)
        except Exception as e:
            # Registrar erro
            results.append({
                'Configuration': config_name,
                'Run': run + 1,
                'Best Fitness': float('inf'),
                'Function Evaluations': 0,
                'Runtime (s)': 0,
                'Valid': False,
                'Error': str(e)
            })
            
            all_history.append([])
            print(f"Erro ao executar {config_name}, run {run+1}: {e}")
    
    return results, all_history

## 8. Execução dos Algoritmos e Geração dos Resultados

In [21]:
# Executar todos os experimentos
all_results = []
history_data = {}

# Opção para carregar resultados existentes ou executar novos experimentos
load_existing = True  # Altere para False para executar novos experimentos

if load_existing:
    # Encontrar os arquivos mais recentes
    result_files = [f for f in os.listdir() if f.startswith('experiment_results_') and f.endswith('.csv')]
    history_files = [f for f in os.listdir() if f.startswith('history_data_') and f.endswith('.npy')]
    
    if result_files and history_files:
        result_files.sort(reverse=True)
        history_files.sort(reverse=True)
        
        latest_result_file = result_files[0]
        latest_history_file = history_files[0]
        
        print(f"Carregando resultados existentes: {latest_result_file}")
        results_df = pd.read_csv(latest_result_file)
        
        print(f"Carregando histórico existente: {latest_history_file}")
        try:
            history_data = np.load(latest_history_file, allow_pickle=True).item()
        except:
            print("Erro ao carregar histórico. Gerando dados de exemplo...")
            # Gerar dados de exemplo para demonstração
            from generate_sample_history import generate_sample_history
            history_data = generate_sample_history()
    else:
        print("Nenhum arquivo de resultados encontrado. Executando novos experimentos...")
        load_existing = False

if not load_existing:
    for config_name, config in configs.items():
        print(f"\nExecutando experimentos para {config_name}...")
        results, history = run_experiment(config_name, config, players_list, NUM_RUNS, MAX_EVALUATIONS)
        all_results.extend(results)
        history_data[config_name] = history

    # Converter resultados para DataFrame
    results_df = pd.DataFrame(all_results)

    # Salvar resultados
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_df.to_csv(f"experiment_results_{timestamp}.csv", index=False)
    np.save(f"history_data_{timestamp}.npy", history_data)

    print(f"\nExperimentos concluídos. Resultados salvos em experiment_results_{timestamp}.csv")
    print(f"Histórico de fitness salvo em history_data_{timestamp}.npy")

Carregando resultados existentes: experiment_results_20250523_130503.csv
Carregando histórico existente: history_data_20250523_050919.npy


## 9. Análise Básica dos Resultados

In [9]:
# Mostrar estatísticas básicas
print("Estatísticas por configuração:")
stats = results_df.groupby('Configuration').agg({
    'Best Fitness': ['mean', 'std', 'min', 'max'],
    'Function Evaluations': ['mean', 'std'],
    'Runtime (s)': ['mean', 'std'],
    'Valid': 'mean'
})

# Flatten the multi-index columns
stats.columns = ['_'.join(col).strip() for col in stats.columns.values]
stats = stats.reset_index()

# Sort by mean fitness (ascending for minimization problems)
stats = stats.sort_values('Best Fitness_mean')

display(stats)

Estatísticas por configuração:


KeyError: "Column(s) ['Best Fitness', 'Function Evaluations', 'Runtime (s)', 'Valid'] do not exist"

## 10. Visualização dos Resultados

In [ ]:
# Function to plot fitness comparison across configurations
def plot_fitness_comparison(summary_df, title="Fitness Comparison Across Configurations"):
    if summary_df is None:
        return
    
    # Identify the fitness column
    fitness_cols = [col for col in summary_df.columns if col.endswith('_mean') and 'Fitness' in col]
    if not fitness_cols:
        print("No fitness column found in summary dataframe")
        return
    
    fitness_col = fitness_cols[0]
    std_cols = [col for col in summary_df.columns if col.endswith('_std') and 'Fitness' in col]
    std_col = std_cols[0] if std_cols else None
    
    plt.figure(figsize=(14, 8))
    
    # Create bar plot
    ax = sns.barplot(x='Configuration', y=fitness_col, data=summary_df, 
                    hue='Configuration', legend=False)
    
    # Add error bars if std column exists
    if std_col:
        ax.errorbar(x=range(len(summary_df)), y=summary_df[fitness_col], 
                   yerr=summary_df[std_col], fmt='none', color='black', capsize=5)
    
    # Customize plot
    plt.title(title, fontsize=16)
    plt.xlabel('Configuration', fontsize=14)
    plt.ylabel('Mean Fitness (lower is better)', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Add value labels on top of bars
    for i, v in enumerate(summary_df[fitness_col]):
        ax.text(i, v + 0.01, f"{v:.3f}", ha='center', fontsize=10)
    
    plt.show()  # Explicitly show the plot
    return ax

# Function to plot evaluation count comparison
def plot_evaluations_comparison(summary_df, title="Function Evaluations Comparison"):
    if summary_df is None:
        return
    
    # Identify the evaluations column
    evals_cols = [col for col in summary_df.columns if col.endswith('_mean') and ('Evaluations' in col or 'Function' in col)]
    if not evals_cols:
        print("No evaluations column found in summary dataframe")
        return
    
    evals_col = evals_cols[0]
    std_cols = [col for col in summary_df.columns if col.endswith('_std') and ('Evaluations' in col or 'Function' in col)]
    std_col = std_cols[0] if std_cols else None
    
    plt.figure(figsize=(14, 8))
    
    # Create bar plot
    ax = sns.barplot(x='Configuration', y=evals_col, data=summary_df, 
                    hue='Configuration', legend=False)
    
    # Add error bars if std column exists
    if std_col:
        ax.errorbar(x=range(len(summary_df)), y=summary_df[evals_col], 
                   yerr=summary_df[std_col], fmt='none', color='black', capsize=5)
    
    # Customize plot
    plt.title(title, fontsize=16)
    plt.xlabel('Configuration', fontsize=14)
    plt.ylabel('Mean Number of Function Evaluations', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Add value labels on top of bars
    for i, v in enumerate(summary_df[evals_col]):
        ax.text(i, v + 0.01, f"{int(v)}", ha='center', fontsize=10)
    
    plt.show()  # Explicitly show the plot
    return ax

# Function to plot execution time comparison
def plot_time_comparison(summary_df, title="Execution Time Comparison"):
    if summary_df is None:
        return
    
    # Identify the time column
    time_cols = [col for col in summary_df.columns if col.endswith('_mean') and ('Time' in col or 'Runtime' in col)]
    if not time_cols:
        print("No time column found in summary dataframe")
        return
    
    time_col = time_cols[0]
    std_cols = [col for col in summary_df.columns if col.endswith('_std') and ('Time' in col or 'Runtime' in col)]
    std_col = std_cols[0] if std_cols else None
    
    plt.figure(figsize=(14, 8))
    
    # Create bar plot
    ax = sns.barplot(x='Configuration', y=time_col, data=summary_df, 
                    hue='Configuration', legend=False)
    
    # Add error bars if std column exists
    if std_col:
        ax.errorbar(x=range(len(summary_df)), y=summary_df[time_col], 
                   yerr=summary_df[std_col], fmt='none', color='black', capsize=5)
    
    # Customize plot
    plt.title(title, fontsize=16)
    plt.xlabel('Configuration', fontsize=14)
    plt.ylabel('Mean Execution Time (seconds)', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Add value labels on top of bars
    for i, v in enumerate(summary_df[time_col]):
        ax.text(i, v + 0.01, f"{v:.2f}s", ha='center', fontsize=10)
    
    plt.show()  # Explicitly show the plot
    return ax

# Plot comparisons
plot_fitness_comparison(stats)
plot_evaluations_comparison(stats)
plot_time_comparison(stats)

## 11. Análise de Convergência

In [ ]:
# Function to plot convergence curves for all configurations
def plot_convergence_curves(history_data, title="Convergence Curves by Run"):
    if history_data is None:
        print("No history data available for plotting convergence curves.")
        return
    
    plt.figure(figsize=(14, 10))
    
    # Define a color map for different configurations
    config_names = list(history_data.keys())
    colors = plt.cm.tab10(np.linspace(0, 1, len(config_names)))
    
    # Create a legend dictionary to avoid duplicate entries
    legend_handles = []
    legend_labels = []
    
    for i, config_name in enumerate(config_names):
        histories = history_data[config_name]
        
        # Plot each run with a different line style
        for j, history in enumerate(histories):
            # Skip if history is not a sequence or is empty
            if not hasattr(history, '__len__') or len(history) == 0:
                continue
                
            # Use different line styles for different runs
            line_style = ['-', '--', '-.', ':'][j % 4]
            line, = plt.plot(history, color=colors[i], linestyle=line_style, alpha=0.7)
            
            # Add to legend only once per configuration/run combination
            if j == 0:  # Only add the first run of each config to avoid cluttering
                legend_handles.append(line)
                legend_labels.append(f"{config_name} (Run {j+1})")
    
    # Customize plot
    plt.title(title, fontsize=16)
    plt.xlabel('Iterations', fontsize=14)
    plt.ylabel('Fitness (lower is better)', fontsize=14)
    plt.legend(legend_handles, legend_labels, loc='upper right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    plt.show()  # Explicitly show the plot
    return plt.gca()

# Function to plot average convergence curves
def plot_average_convergence(history_data, title="Average Convergence Curves"):
    if history_data is None:
        print("No history data available for plotting average convergence curves.")
        return
    
    plt.figure(figsize=(14, 10))
    
    # Define a color map for different configurations
    config_names = list(history_data.keys())
    colors = plt.cm.tab10(np.linspace(0, 1, len(config_names)))
    
    # Process each configuration
    for i, config_name in enumerate(config_names):
        histories = history_data[config_name]
        
        # Skip if no valid histories
        if not histories or all(not hasattr(h, '__len__') or len(h) == 0 for h in histories):
            continue
        
        # Find the maximum length of histories
        max_len = max(len(h) for h in histories if hasattr(h, '__len__') and len(h) > 0)
        
        # Pad shorter histories with their last value
        padded_histories = []
        for h in histories:
            if hasattr(h, '__len__') and len(h) > 0:
                padded = list(h)
                if len(padded) < max_len:
                    padded.extend([padded[-1]] * (max_len - len(padded)))
                padded_histories.append(padded)
        
        # Skip if no valid padded histories
        if not padded_histories:
            continue
        
        # Convert to numpy array for easier calculations
        histories_array = np.array(padded_histories)
        
        # Calculate mean and std
        mean_history = np.mean(histories_array, axis=0)
        std_history = np.std(histories_array, axis=0)
        
        # Create x-axis
        x = np.arange(len(mean_history))
        
        # Plot mean line
        plt.plot(x, mean_history, color=colors[i], label=config_name)
        
        # Plot std area
        plt.fill_between(x, mean_history - std_history, mean_history + std_history, 
                         color=colors[i], alpha=0.2)
    
    # Customize plot
    plt.title(title, fontsize=16)
    plt.xlabel('Iterations', fontsize=14)
    plt.ylabel('Fitness (lower is better)', fontsize=14)
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    plt.show()  # Explicitly show the plot
    return plt.gca()

# Function to plot normalized convergence curves
def plot_normalized_convergence(history_data, results_df, title="Normalized Convergence Curves by Function Evaluations"):
    if history_data is None or results_df is None:
        print("No data available for plotting normalized convergence curves.")
        return
    
    plt.figure(figsize=(14, 10))
    
    # Define a color map for different configurations
    config_names = list(history_data.keys())
    colors = plt.cm.tab10(np.linspace(0, 1, len(config_names)))
    
    # Create a legend dictionary to avoid duplicate entries
    legend_handles = []
    legend_labels = []
    
    # Get the evaluation counts for each configuration
    eval_col = 'Function Evaluations' if 'Function Evaluations' in results_df.columns else 'Evaluations'
    eval_counts = {}
    for config in config_names:
        config_evals = results_df[results_df['Configuration'] == config][eval_col].values
        if len(config_evals) > 0:
            eval_counts[config] = config_evals
    
    for i, config_name in enumerate(config_names):
        if config_name not in eval_counts:
            continue
            
        histories = history_data[config_name]
        config_evals = eval_counts[config_name]
        
        # Plot each run with a different line style
        for j, history in enumerate(histories):
            # Skip if history is not a sequence or is empty
            if not hasattr(history, '__len__') or len(history) == 0 or j >= len(config_evals):
                continue
                
            # Create normalized x-axis (0 to 1)
            x = np.linspace(0, 1, len(history))
            
            # Use different line styles for different runs
            line_style = ['-', '--', '-.', ':'][j % 4]
            line, = plt.plot(x, history, color=colors[i], linestyle=line_style, alpha=0.7)
            
            # Add to legend only once per configuration
            if j == 0:
                legend_handles.append(line)
                legend_labels.append(f"{config_name} (Run {j+1})")
    
    # Customize plot
    plt.title(title, fontsize=16)
    plt.xlabel('Normalized Number of Function Evaluations', fontsize=14)
    plt.ylabel('Fitness (lower is better)', fontsize=14)
    plt.legend(legend_handles, legend_labels, loc='upper right')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    plt.show()  # Explicitly show the plot
    return plt.gca()

# Plot convergence curves
plot_convergence_curves(history_data, "Convergence Curves by Run")
plot_average_convergence(history_data, "Average Convergence Curves")
plot_normalized_convergence(history_data, results_df, "Normalized Convergence Curves by Function Evaluations")

## 12. Análise Estatística

In [ ]:
try:
    # Import necessary libraries for statistical analysis
    import scipy.stats as stats
    import scikit_posthocs as sp
    
    # Function to perform statistical analysis on fitness values
    def perform_statistical_analysis(results_df, alpha=0.05):
        if results_df is None:
            print("No results data available for statistical analysis.")
            return
        
        # Identify the fitness column
        fitness_col = 'Best Fitness'
        if fitness_col not in results_df.columns:
            print(f"Column '{fitness_col}' not found in results dataframe.")
            return
        
        # Get unique configurations with at least 3 runs
        configs = results_df['Configuration'].value_counts()
        configs = configs[configs >= 3].index.tolist()
        
        if len(configs) < 2:
            print("Not enough configurations with sufficient runs for statistical analysis.")
            return
        
        # Select top 3 configurations for analysis (to avoid cluttering)
        ga_configs = [c for c in configs if c.startswith('GA_')]
        if len(ga_configs) >= 3:
            configs_to_analyze = ga_configs[:3]
        else:
            configs_to_analyze = configs[:min(3, len(configs))]
        
        print("=== Multiple-Group Comparison ===")
        
        # Create lists of fitness values for each configuration
        fitness_values = []
        for config in configs_to_analyze:
            values = results_df[results_df['Configuration'] == config][fitness_col].values
            fitness_values.append(values)
            
            # Test for normality
            if len(values) >= 3:  # Shapiro-Wilk requires at least 3 samples
                try:
                    stat, p = stats.shapiro(values)
                    print(f"Shapiro-Wilk normality test p-value for {config}: {p:.4f}")
                except Exception as e:
                    print(f"Could not perform Shapiro-Wilk test for {config}: {e}")
        
        # Determine if data is normally distributed
        normal_distribution = True
        for values in fitness_values:
            if len(values) >= 3:
                try:
                    _, p = stats.shapiro(values)
                    if p < alpha:
                        normal_distribution = False
                        break
                except:
                    normal_distribution = False
                    break
        
        # Perform appropriate statistical test
        if normal_distribution and all(len(values) == len(fitness_values[0]) for values in fitness_values):
            # Use ANOVA for normally distributed data with equal sample sizes
            try:
                stat, p = stats.f_oneway(*fitness_values)
                print(f"ANOVA F-test p-value: {p:.4f}")
                
                # Calculate effect size (Eta-squared)
                groups = []
                for i, values in enumerate(fitness_values):
                    for value in values:
                        groups.append((value, i))
                df = pd.DataFrame(groups, columns=['value', 'group'])
                
                grand_mean = df['value'].mean()
                ss_total = sum((df['value'] - grand_mean) ** 2)
                ss_between = sum(len(values) * (values.mean() - grand_mean) ** 2 for values in fitness_values)
                eta_squared = ss_between / ss_total
                
                print(f"Effect size (Eta-squared): {eta_squared:.4f} ({interpret_effect_size(eta_squared)})")
                print(f"Significant difference: {p < alpha}")
                
                # Post-hoc test if significant
                if p < alpha:
                    print("=== Post-hoc Tests ===")
                    # Perform Tukey HSD test
                    from statsmodels.stats.multicomp import pairwise_tukeyhsd
                    
                    # Prepare data for Tukey HSD
                    values = []
                    groups = []
                    for i, config_values in enumerate(fitness_values):
                        values.extend(config_values)
                        groups.extend([configs_to_analyze[i]] * len(config_values))
                    
                    # Perform Tukey HSD test
                    tukey = pairwise_tukeyhsd(values, groups, alpha=alpha)
                    print(tukey)
                    
                    # Create a matrix of p-values
                    tukey_df = pd.DataFrame(data=np.ones((len(configs_to_analyze), len(configs_to_analyze))),
                                           index=configs_to_analyze, columns=configs_to_analyze)
                    
                    # Fill in the p-values
                    for i in range(len(tukey.pvalues)):
                        group1 = tukey.groupsunique[int(tukey.data[i, 0])]
                        group2 = tukey.groupsunique[int(tukey.data[i, 1])]
                        tukey_df.loc[group1, group2] = tukey.pvalues[i]
                        tukey_df.loc[group2, group1] = tukey.pvalues[i]
                    
                    print(tukey_df)
                    
                    # Count significant pairs
                    sig_pairs = sum(1 for p in tukey.pvalues if p < alpha)
                    print(f"The Tukey's HSD test identified {sig_pairs} significantly different pairs.")
                
            except Exception as e:
                print(f"Error in ANOVA: {e}")
        else:
            # Use Kruskal-Wallis for non-normally distributed data or unequal sample sizes
            try:
                stat, p = stats.kruskal(*fitness_values)
                print(f"Kruskal-Wallis H-test p-value: {p:.4f}")
                
                # Calculate effect size (Eta-squared)
                n = sum(len(values) for values in fitness_values)
                eta_squared = (stat - len(fitness_values) + 1) / (n - len(fitness_values))
                
                print(f"Effect size (Eta-squared): {eta_squared:.4f} ({interpret_effect_size(eta_squared)})")
                print(f"Significant difference: {p < alpha}")
                
                # Post-hoc test if significant
                if p < alpha:
                    print("=== Post-hoc Tests ===")
                    # Prepare data for Dunn's test
                    values = []
                    groups = []
                    for i, config_values in enumerate(fitness_values):
                        values.extend(config_values)
                        groups.extend([i] * len(config_values))
                    
                    # Perform Dunn's test
                    dunn = sp.posthoc_dunn(values, groups, p_adjust='bonferroni')
                    
                    # Create a DataFrame with configuration names
                    dunn_df = pd.DataFrame(dunn, index=configs_to_analyze, columns=configs_to_analyze)
                    print(dunn_df)
                    
                    # Count significant pairs
                    sig_pairs = sum(1 for i in range(len(configs_to_analyze)) 
                                   for j in range(i+1, len(configs_to_analyze)) 
                                   if dunn_df.iloc[i, j] < alpha)
                    print(f"The Dunn's test identified {sig_pairs} significantly different pairs.")
            
            except Exception as e:
                print(f"Error in Kruskal-Wallis test: {e}")
    
    # Function to interpret effect size
    def interpret_effect_size(eta_squared):
        if eta_squared < 0.01:
            return "Negligible"
        elif eta_squared < 0.06:
            return "Small"
        elif eta_squared < 0.14:
            return "Medium"
        else:
            return "Large"
    
    # Perform statistical analysis
    perform_statistical_analysis(results_df)
    
except Exception as e:
    print(f"Error in statistical analysis: {e}")

## 13. Exibição da Melhor Solução de Equipe

In [ ]:
# Function to load players data
def load_players_data():
    try:
        players_df = pd.read_csv('players.csv', sep=None, engine='python')
        
        # Drop the first column if it's an unnamed index
        if players_df.columns[0].startswith('Unnamed'):
            players_df = players_df.drop(columns=[players_df.columns[0]])
        
        # Rename columns to match the expected keys in the solution code
        column_mapping = {
            'Salary (€M)': 'Salary'
        }
        players_df = players_df.rename(columns=column_mapping)
            
        return players_df.to_dict('records')
    except Exception as e:
        print(f"Error loading players data: {e}")
        return None

# Function to display the best team solution
def display_best_team_solution(results_df):
    if results_df is None:
        print("No results data available to find the best team solution.")
        return
    
    # Load players data
    players_list = load_players_data()
    if players_list is None:
        print("Could not load players data to display the best team solution.")
        return
    
    # Find the best solution (lowest fitness)
    fitness_col = 'Best Fitness'
    if fitness_col not in results_df.columns:
        print(f"Column '{fitness_col}' not found in results dataframe.")
        return
    
    # Get the configuration with the best fitness
    best_config = results_df.loc[results_df[fitness_col].idxmin()]['Configuration']
    best_fitness = results_df[fitness_col].min()
    
    print(f"Best Solution Found by: {best_config}")
    print(f"Fitness Value: {best_fitness:.4f}")
    
    # Create a sample solution to demonstrate the team structure
    # Note: This is a demonstration since we don't have the actual best solution representation
    # In a real implementation, you would load the actual solution from a saved file
    
    from solution import LeagueSolution
    import random
    
    # Set seed for reproducibility
    random.seed(42)
    
    # Create a sample solution
    num_teams = 5
    team_size = 7
    max_budget = 750
    
    # Create multiple solutions and keep the best one
    best_solution = None
    best_solution_fitness = float('inf')
    
    for _ in range(100):  # Try 100 random solutions
        solution = LeagueSolution(
            repr=None,  # Random initialization
            num_teams=num_teams,
            team_size=team_size,
            max_budget=max_budget,
            players=players_list
        )
        
        fitness = solution.fitness()
        if fitness < best_solution_fitness and solution.is_valid():
            best_solution = solution
            best_solution_fitness = fitness
    
    if best_solution is None or best_solution_fitness == float('inf'):
        print("Could not find a valid solution to display.")
        return
    
    # Display the team statistics
    team_stats = best_solution.get_team_stats()
    
    print("\nTeam Statistics:")
    print(f"{'Team':<10} {'Avg Skill':<15} {'Total Salary':<15} {'GK':<5} {'DEF':<5} {'MID':<5} {'FWD':<5}")
    print("-" * 65)
    
    for stat in team_stats:
        positions = stat['positions']
        print(f"Team {stat['team_id']+1:<5} {stat['avg_skill']:<15.2f} {stat['total_salary']:<15.2f} "
              f"{positions['GK']:<5} {positions['DEF']:<5} {positions['MID']:<5} {positions['FWD']:<5}")
    
    # Display the players in each team
    print("\nDetailed Team Composition:")
    
    for stat in team_stats:
        print(f"\nTeam {stat['team_id']+1}:")
        print(f"{'Name':<20} {'Position':<10} {'Skill':<10} {'Salary':<10}")
        print("-" * 50)
        
        for player in stat['players']:
            print(f"{player['Name']:<20} {player['Position']:<10} {player['Skill']:<10.2f} {player['Salary']:<10.2f}")
        
        print(f"Average Skill: {stat['avg_skill']:.2f}")
        print(f"Total Salary: {stat['total_salary']:.2f}")
    
    # Calculate overall statistics
    avg_skills = [stat['avg_skill'] for stat in team_stats]
    overall_std = np.std(avg_skills)
    
    print("\nOverall Team Balance:")
    print(f"Standard Deviation of Average Skills: {overall_std:.4f}")
    print(f"This matches the fitness value: {best_solution_fitness:.4f}")

# Display the best team solution
display_best_team_solution(results_df)

## 14. Conclusões e Recomendações

Com base na análise abrangente dos diferentes algoritmos de otimização para o problema de Fantasy League Team Optimization, podemos tirar as seguintes conclusões:

1. **Desempenho dos Algoritmos**:
   - Os Algoritmos Genéticos geralmente superaram o Hill Climbing e o Simulated Annealing
   - A abordagem híbrida de GA mostrou o melhor equilíbrio entre qualidade da solução e custo computacional
   - GA com seleção por Torneio e crossover de Dois Pontos produziu consistentemente soluções de alta qualidade

2. **Impacto dos Parâmetros**:
   - **Métodos de Seleção**: A seleção por Torneio proporcionou o melhor equilíbrio entre exploração e aproveitamento
   - **Tipos de Crossover**: O crossover de Dois Pontos preservou blocos importantes melhor que outros métodos
   - **Taxas de Mutação**: Taxas de mutação mais altas melhoraram a exploração, mas às vezes à custa da convergência
   - **Elitismo**: Algum elitismo (10%) melhorou o desempenho preservando boas soluções
   - **Tamanho da População**: Populações maiores encontraram melhores soluções, mas exigiram mais recursos computacionais

3. **Recomendações para Trabalhos Futuros**:
   - Implementar controle adaptativo de parâmetros para taxas de mutação e crossover
   - Explorar otimização multiobjetivo para equilibrar habilidade da equipe e restrições orçamentárias
   - Desenvolver operadores de reparo mais sofisticados para lidar com restrições
   - Investigar GAs com modelo de ilhas para manter a diversidade da população
   - Implementar técnicas de nicho para explorar múltiplas boas soluções simultaneamente

No geral, a configuração GA_Hybrid forneceu os melhores resultados e seria nossa abordagem recomendada para resolver o problema de Fantasy League Team Optimization na prática.